In [2]:
import requests
import requests.auth
import praw
import re
import emoji
from textblob import TextBlob
import pandas as pd


# user information and auth info

user_id = 'kaythomedu'
user_pass = None
client_id = 'Ww1gfcX500dqWg'
secret = 'YiQCz4XAinioTTO0Xczzk5lITV25tQ'
user_agent = 'User-Agent:com.example.politicsresearch:v1.0.0 (by u/kaythomedu)'

client_auth = requests.auth.HTTPBasicAuth(client_id, secret)

# initializing Reddit 
reddit = praw.Reddit(
    client_id=client_id,
    client_secret=secret,
    user_agent=user_agent)

# Cleaning
def clean(x):
    x = re.sub(r'https?:\/\/.*[\r\n]*', '', x)
    x = re.sub(r'#', '', x) # remove hashtag
    x = re.sub(r'u/[A-Za-z0-9]+', '', x) # remove reference to other users
    x = re.sub(r'\*', '', x)
    x = emoji.replace_emoji(x, '')
    return x

In [3]:
# assign subreddit
subreddit = reddit.subreddit('politics')

# searching for top posts within the last year that match the search query on the assigned subreddit
counter = 0
all_comms = []
posts = []
for post in subreddit.top(time_filter='year'):
    print(str(post.id) + '  ' + str(post.created_utc) + '  ' + str(post.title) + '\n')
    post.comment_sort = 'top'
    post.comments.replace_more(limit=None)
    posts.append((post.id, post.title))
    # collecting and cleaning comments from each post
    all_comments = []
    for init_comment in post.comments.list():
        comm_body = clean(init_comment.body)
        blob_text = TextBlob(comm_body)
        all_comments.append((init_comment.author, 
        init_comment.id, 
        init_comment.body, 
        init_comment.link_id, #link id tracks the post a user comments on
        blob_text.polarity)) # polarity is positive or negative sentiment, -1 negative, +1 postitive
    print(len(all_comments))
    all_comms.append(all_comments)
    counter += 1
    if counter == 50:
        break

nk564e  1621882122.0  Sen. Elizabeth Warren wants to bar members of Congress from ever trading individual stocks again

3205
t2suj2  1645982118.0  Mitt Romney says Americans who support Putin are ‘almost treasonous’

5342
t11uq0  1645788662.0  Trump, who was impeached for withholding nearly $400 million in military aid from Ukraine, said 'this deadly Ukraine situation would never have happened' if he were in office

6453
rxqvwi  1641508272.0  Three in four voters support banning lawmakers from trading stocks: poll

2585
su3zob  1645039370.0  Sen. Elizabeth Warren now wants to ban all state lawmakers and elected officials from trading corporate stocks

3012
tp4xtx  1648331255.0  President Joe Biden to propose new 20% minimum billionaire tax

6875
s739ob  1642528155.0  67% of Americans support bill to ban stock trading by members of Congress, even as Pelosi resists

2333
ubo2ao  1650901335.0  Complaint filed in Florida to ban Bible as “too woke” | The book contains several "prohibited to

In [4]:
# preparing for dataframe

post_id = []
post_title = []

for id, title in posts:        
    post_id.append(id)
    post_title.append(title)

orig_post_info = {'post_id' : post_id, 'title' : post_title}

author = []
author_id = []
comment_text = []
orig_post_id = []
sentiment = []
for comm in all_comms:
    for author_name, id, body, post_id, sentiment_score in comm:
        author.append(author_name)
        author_id.append(id) 
        comment_text.append(body) 
        sentiment.append(sentiment_score)
        orig_post_id.append(post_id) 

commenter_info = {
    'author' : author, 
    'author_id' : author_id, 
    'comment' : comment_text, 
    'sentiment' : sentiment, 
    'post_id' : orig_post_id}

# create dataframes and save as csv
orig_post_df = pd.DataFrame(orig_post_info)
all_comms_df = pd.DataFrame(commenter_info)

orig_post_df.to_csv('reddit_posts.csv')
all_comms_df.to_csv('reddit_comments.csv')

In [15]:
filtered_comms_df = all_comms_df[(all_comms_df.sentiment<=-0.5) | (all_comms_df.sentiment>=0.5)]
filtered_comms_df

,author,author_id,comment,sentiment,post_id
2,Fuzzy_darkman,gzaxuhh,"No way it will pass, but glad she's pushing th...",0.5,t3_nk564e
9,cynicalhysteria,gzb3zcm,Such a wonderful idea. How many members of Con...,0.5,t3_nk564e
10,GrilledStuffedDragon,gzaxj5v,Good.,0.7,t3_nk564e
17,temporvicis,gzaxocd,Sounds like a great idea. She should run for ...,0.8,t3_nk564e
29,bust-the-shorts,gzb4yu8,It’s a good start.,0.7,t3_nk564e
...,...,...,...,...,...
215554,Glenncoco23,i2kgxvl,I also talked about how the article was alread...,-0.5,t3_tqxa6i
215565,None,i2nsvdk,There’s nothing wrong with what he said.,-0.5,t3_tqxa6i
215577,None,i2li24x,Asking to find votes is not a crime. If he wou...,-0.5,t3_tqxa6i
215590,limitedkp,i2lywph,…the case is ongoing lol,0.8,t3_tqxa6i


In [13]:
filtered_comms_df.to_csv('reddit_comments_filtered.csv')

In [20]:
filtered_comms_df.duplicated(subset=['author']).value_counts()

False    17567
True      4674
dtype: int64

In [40]:
filtered_comms_df[filtered_comms_df.duplicated(subset=['author']) & (~filtered_comms_df.author.isnull())].to_csv('reddit_comments_filtered_2.csv')

,author,author_id,comment,sentiment,post_id
497,mbleslie,gzbohk3,Counterpoint: without ability to own/trade sto...,0.5,t3_nk564e
1569,tomjoads,gzj6hj3,Once transferred the charity they would no lon...,0.5,t3_nk564e
1678,BennoWutzi1954,gzf6bz4,What about Mitch? He's filthy and rich.where d...,-0.8,t3_nk564e
1716,Fuzzy_darkman,gzbd8g8,Oh I'm sure there will be.,0.5,t3_nk564e
1724,Fuzzy_darkman,gzc6xvc,Which is why the GOP will make damn sure it do...,0.5,t3_nk564e
...,...,...,...,...,...
215475,_United_,i2nq6jq,i believe you're replying to the wrong person,-0.5,t3_tqxa6i
215519,_United_,i2qf2fj,you can just say you don't want to read anythi...,0.5,t3_tqxa6i
215523,Razakel,i2limc8,They're all dictators of countries that were n...,-0.5,t3_tqxa6i
215554,Glenncoco23,i2kgxvl,I also talked about how the article was alread...,-0.5,t3_tqxa6i


In [56]:
filtered_comms_df[filtered_comms_df['author'].duplicated(keep=False) & (~filtered_comms_df.author.isnull())].to_csv('reddit_comments_filtered_2.csv')